In [9]:
import pandas as pd
import numpy as np

def import_data():
    data = pd.read_csv('https://supchains.com/wp-content/uploads/2021/07/norway_new_car_sales_by_make1.csv') #import data
    data['Period'] = data['Year'].astype(str) + '-' + data['Month'].astype(str).str.zfill(2) #create period column and account for padded zeros
    df = pd.pivot_table(data = data, values='Quantity', index = 'Make', columns = 'Period', aggfunc = 'sum', fill_value=0)
    return df

def datasets(df, x_len = 12, y_len = 1, test_loops = 12):
    D = df.values
    rows,periods = df.shape
    
    #training set creation
    loops = periods + 1 - x_len - y_len
    train = []
    for col in range(loops):
        train.append(D[:,col:col+x_len+y_len])
    train = np.vstack(train)
    X_train, Y_train = np.split(train, [-y_len], axis = 1)

    #test set creation
    if test_loops > 0:
        X_train, X_test = np.split(X_train, [-rows*test_loops], axis = 0)
        Y_train, Y_test = np.split(Y_train, [-rows*test_loops], axis = 0)
    else: #no test set: X_test is used to generate the future forecast
        X_test = D[:,-x_len:]
        Y_test = np.full((X_test.shape[0],y_len), np.nan) #dummy value

    #formatting required for scikit-learn
    if y_len == 1:
        Y_train = Y_train.ravel()
        Y_test = Y_test.ravel()

    return(X_train, Y_train, X_test, Y_test)
    

def kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name=''):
    df = pd.DataFrame(columns = ['MAE', 'RMSE', 'Bias'], index = ['Train', 'Test'])
    df.index.name = name

    df.loc['Train','MAE'] = 100*np.mean(abs(Y_train - Y_train_pred))/np.mean(Y_train)

    df.loc['Train','RMSE'] = 100*np.sqrt(np.mean((Y_train - Y_train_pred)**2))/np.mean(Y_train)

    df.loc['Train','Bias'] = 100*np.mean(abs(Y_train - Y_train_pred))/np.mean(Y_train)

    df.loc['Test','MAE'] = 100*np.mean(abs(Y_test - Y_test_pred))/np.mean(Y_test)

    df.loc['Test','RMSE'] = 100*np.sqrt(np.mean((Y_test - Y_test_pred)**2))/np.mean(Y_test)

    df.loc['Test','Bias'] = 100*np.mean(abs(Y_test - Y_test_pred))/np.mean(Y_test)

    df = df.astype(float).round(1)
    print(df)

df = import_data()
X_train, Y_train, X_test, Y_test = datasets(df, x_len=12, y_len=1, test_loops=12)

from sklearn.linear_model import LinearRegression
reg = LinearRegression() #create linear regression object
reg = reg.fit(X_train, Y_train) #fit it to training data

#create two predictions for the training and test sets
Y_train_pred = reg.predict(X_train)
Y_test_pred = reg.predict(X_test)

kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name='Regression')


X_train, Y_train, X_test, Y_test = datasets(df, x_len=12, y_len=1, test_loops =0)
reg = LinearRegression()
reg = reg.fit(X_train, Y_train)
forecast = pd.DataFrame(data = reg.predict(X_test), index = df.index)
forecast.head()

 

             MAE  RMSE  Bias
Regression                  
Train       17.8  43.9  17.8
Test        17.8  43.7  17.8


,0
Make,
Alfa Romeo,6.187217
Aston Martin,1.032483
Audi,646.568622
BMW,1265.032834
Bentley,1.218092


In [10]:
df

Period,2007-01,2007-02,2007-03,2007-04,2007-05,2007-06,2007-07,2007-08,2007-09,2007-10,...,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01
Make,,,,,,,,,,,,,,,,,,,,,
Alfa Romeo,16,9,21,20,17,21,14,12,15,10,...,3,1,2,1,6,15,3,4,3,6
Aston Martin,0,0,1,0,4,3,3,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Audi,599,498,682,556,630,498,562,590,393,554,...,685,540,551,687,794,688,603,645,827,565
BMW,352,335,365,360,431,477,403,348,271,562,...,1052,832,808,636,1031,1193,1096,1663,866,1540
Bentley,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Think,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Toyota,2884,1885,1833,1300,1866,1620,1901,1783,1303,1648,...,1432,1687,1603,1127,1824,1823,1327,1375,1238,1526
Volkswagen,2521,1517,1428,1257,1934,1531,1777,1665,1373,1941,...,3017,2222,2287,2076,2359,2084,2161,2106,2239,1688


In [11]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(max_depth = 5, min_samples_split = 15, min_samples_leaf = 5)

tree.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=5, min_samples_split=15)

In [12]:
Y_train_pred = tree.predict(X_train)
Y_test_pred = tree.predict(X_test)
kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name = 'Tree')

        MAE  RMSE  Bias
Tree                   
Train  18.2  44.1  18.2
Test    NaN   NaN   NaN


In [14]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (15,6), dpi=300)
ax = fig.gca()
plot_tree(tree, fontsize = 3, feature_names = [f'M{x-12}' for x in range(12)], rounded = True, filled= True, ax=ax)
fig.savefig('Regression Tree.PNG')